# Custom Updater

## Overview

### Questions

- How can I model the bombardment of radiation into my system?

### Objectives

- Show an example of a non-trival custom updater.

## Boilerplate Code

In [1]:
from numbers import Number

import numpy as np

import hoomd
import hoomd.md as md


cpu = hoomd.device.CPU()
sim = hoomd.Simulation(cpu)

# Create a simple cubic configuration of particles
N = 5  # particles per box direction
box_L = 20  # box dimension

snap = hoomd.Snapshot(cpu.communicator)
snap.configuration.box = [box_L] * 3 + [0, 0, 0]
snap.particles.N = N ** 3
x, y, z = np.meshgrid(
    *(np.linspace(-box_L / 2, box_L / 2, N, endpoint=False),) * 3)
positions = np.array((x.ravel(), y.ravel(), z.ravel())).T
snap.particles.position[:] = positions
snap.particles.types = ['A']
snap.particles.typeid[:] = 0

sim.create_state_from_snapshot(snap)

## Problem

In this section, we will create a custom updater that adds a
prescribed amount of energy to a single particle simulating
the bombardment of radioactive material into our system. For
this problem, we need to pick a random particle and modify
its velocity according to the radiation energy in a random 
direction.

In [2]:
class InsertEnergyUpdater(hoomd.custom.Action):
    def __init__(self, energy):
        self.energy = energy
    
    def act(self, timestep):
        snap = self._state.snapshot
        if snap.exists:
            particle_i = np.random.randint(snap.particles.N)
            mass = snap.particles.mass[particle_i]
            direction = self._get_direction()
            magnitude = np.sqrt(2 * self.energy / mass)
            velocity = direction * magnitude
            old_velocity = snap.particles.velocity[particle_i]
            new_velocity = old_velocity + velocity
            snap.particles.velocity[particle_i] = velocity
        self._state.snapshot = snap
            
    @staticmethod
    def _get_direction():
        theta, z = np.random.rand(2)
        theta *= 2 * np.pi
        z = 2 * (z - 0.5)
        return np.array([
            np.sqrt(1 - (z * z)) * np.cos(theta),
            np.sqrt(1 - (z * z)) * np.sin(theta),
            z
        ])

Define a function that creates a `InsertEnergyUpdater` wrapped in a custom updater.

In [3]:
def create_insert_energy_updater(trigger, *args, **kwargs):
    return hoomd.update.CustomUpdater(
        action=InsertEnergyUpdater(*args, **kwargs),
        trigger=trigger)

We will now use our custom updater with an `NVE` integrator.
Particles will interact via a Lennerd-Jones potential.
Using the `Table` writer and a `hoomd.logging.Logger`, we will
monitor the energy which should be increasing as we are
inserting energy into the system. We will also thermalize our
system to a `kT == 1`.

In [4]:
sim.state.thermalize_particle_momenta(hoomd.filter.All(), 1., seed=109)

lj = md.pair.LJ(nlist=md.nlist.Cell())
lj.params[('A', 'A')] = {'epsilon': 1.,
                         'sigma': 1.}
lj.r_cut[('A', 'A')] = 2.5
integrator = md.Integrator(methods=[md.methods.NVE(hoomd.filter.All())],
                           forces=[lj],
                           dt=0.005)

thermo = md.compute.ThermodynamicQuantities(hoomd.filter.All())
logger = hoomd.logging.Logger(categories=['scalar'])
logger.add(thermo, ['kinetic_energy', 'potential_energy'])
logger['total_energy'] = (
    lambda: thermo.kinetic_energy + thermo.potential_energy,
    'scalar')

table = hoomd.write.Table(100, logger, max_header_len=1)

sim.operations += integrator
sim.operations += thermo
sim.operations += table

# Create and add our custom updater
energy_inserter = create_insert_energy_updater(
    trigger=100, energy=10.)

sim.operations += energy_inserter

In [5]:
sim.run(1000)

 kinetic_energy  potential_energy   total_energy  
   201.47616         -0.04328        201.43288    
   210.47491         -3.15911        207.31580    
   223.81249         -8.14532        215.66717    
   231.07897         -7.54589        223.53309    
   242.60525         -9.06074        233.54450    
   246.71857        -12.70806        234.01051    
   254.02928        -12.37860        241.65069    
   258.43691         -9.89064        248.54626    
   268.42864        -10.59255        257.83608    
   274.10669        -10.87363        263.23305    


As we can see the total energy of the system is indeed increasing.
The energy isn't increasing by 10 every time since we are adding
the velocity in a random direction which may be against the current
velocity.

Maybe we want to allow for the energy to be from a distribution.
HOOMD-blue has a concept called a variant which allows for quantities
that vary over time. Let's change the `InsertEnergyupdater` to use
variants and create a custom variant that grabs a random number from
a Gaussian distribution. (If you don't understand the variant code,
that is fine. We are using this to just showcase how you can iterative
improve custom actions).

In [6]:
class InsertEnergyUpdater(hoomd.custom.Action):
    def __init__(self, energy):
        self._energy = energy
        
    @property
    def energy(self):
        return self._energy
    
    @energy.setter
    def energy(self, new_energy):
        if isinstance(new_energy, Number):
            self._energy = hoomd.variant.Constant(new_energy)
        elif isinstance(new_energy, hoomd.variant.Variant):
            self._energy = new_energy
        else:
            raise ValueError(
                "energy must be a variant or real number.")
    
    def act(self, timestep):
        snap = self._state.snapshot
        if snap.exists:
            particle_i = np.random.randint(snap.particles.N)
            mass = snap.particles.mass[particle_i]
            direction = self._get_direction()
            magnitude = np.sqrt(2 * self.energy(timestep) / mass)
            velocity = direction * magnitude
            old_velocity = snap.particles.velocity[particle_i]
            new_velocity = old_velocity + velocity
            snap.particles.velocity[particle_i] = velocity
        self._state.snapshot = snap
            
    @staticmethod
    def _get_direction():
        theta, z = np.random.rand(2)
        theta *= 2 * np.pi
        z = 2 * (z - 0.5)
        return np.array([
            np.sqrt(1 - (z * z)) * np.cos(theta),
            np.sqrt(1 - (z * z)) * np.sin(theta),
            z
        ])


class GaussianVariant(hoomd.variant.Variant):
    def __init__(self, mean, std):
        hoomd.variant.Variant.__init__(self)
        self.mean = mean
        self.std = std
    
    def __call__(self, timestep):
        return np.random.normal(self.mean, self.std)

We will now use this variant and the improved updater in the 
simulation. We will also show that the Gaussian Variant works.

In [7]:
energy = GaussianVariant(10., 2.)
sample_energies = np.array([energy(0) for _ in range(1000)])
display(sample_energies.mean(), sample_energies.std())
sample_energies[:10]

10.083418012511315

1.9842429842116385

array([13.52435343, 11.49635971, 11.00686198, 10.59921007, 11.05856711,
       11.17795347, 10.06254476, 10.04700913, 10.67985303, 10.38650408])

In [8]:
sim.operations.updaters.remove(energy_inserter)
energy_inserter = create_insert_energy_updater(
    100, energy=energy)
sim.operations.updaters.append(energy_inserter)
sim.run(1000)

   268.92137         -7.47786        261.44351    
   276.93600         -9.04181        267.89419    
   289.83069         -9.99470        279.83600    
   292.48113        -10.84733        281.63379    
   301.03107        -11.09542        289.93565    
   305.67430        -11.22536        294.44894    
   310.70524        -11.03718        299.66806    
   315.88697         -8.74890        307.13806    
   330.22740        -11.86236        318.36505    
   336.27599        -12.10039        324.17560    


We could continue to improve upon this updater and the execution of
this operation. However, this suffices to showcase the ability of non-trivial updaters to effect simulation state.